In [34]:
#install packages
from gettext import npgettext
from importlib.metadata import requires
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [36]:
#import url
url = 'https://kworb.net/spotify/country/hk_daily.html'

# using requests package to download from web
req = requests.get(url)
req.encoding = 'utf-8' # super important!!!
assert req
# <Response [200]> meaning it is successful

soup = BeautifulSoup (req.text, features ='html.parser')
#print(soup.prettify())  

#obtain info. from tag <body>
table1 = soup.find('table',id='spotifydaily')
assert table1

In [29]:
# Obtain every title of columns with tag <th>
# <table><tr><th>ABC</th><th>DEF</th></table>
# Transformation / Mapping
# ["ABC", "DEF"]
# ["ABC1234", "DE3415"] -> ["ABC", "DE"] (map)
# [1, 5, 1, 2, 5, 6, 1] -> 21 (reduce)
# reduce(lambda acc, x: acc+x, array, 0)

headers = list( 
  map(
    lambda x: x.text, 
    table1.find_all('th')
  ) 
)

# headers=[]
# for th_cell in table1.find_all('th'):
#   title = th_cell.text
#   headers.append(title)

assert headers

In [30]:
#create dataframe, adding columns inside
mydata = pd.DataFrame (columns = headers)

mydata


,Pos,P+,Artist and Title,Days,Pk,(x?),Streams,Streams+,7Day,7Day+,Total


In [31]:
# mydata
# Iteration 1: mydata = [], length = 0, mydata.loc[0] = ....
# Iteration 2: mydata = [....], length = 1, mydata.loc[1] = yyyy
# Iteration 3: mydata = [....,yyyy], length = 2, mydata.loc[2] = xxxx

#create a for loop to full mydata
for row in table1.find_all('tr')[1:]:  # j 做variable, 搵 <tr> 由第2行去到最尾
    cells = row.find_all('td') # 每一個j 搵入面<td>所有野，做row_data
    cell_text = [ i.text for i in cells ]
    pos = len(mydata)
    mydata.loc[pos] = cell_text


# for line in mydata:
#     print (line.encode('utf-8'))

print(mydata)

     Pos   P+                                  Artist and Title  Days   Pk  \
0      1    =                                      MC 張天賦 - 記憶棉    94    1   
1      2    =                                     MC 張天賦 - 反對無效   234    2   
2      3    =                                     MC 張天賦 - 時候不早    41    2   
3      4    =                            COLLAR - Call My Name!    11    1   
4      5    =                    William Wei - 如果可以 - 電影"月老"主題曲    48    4   
..   ...  ...                                               ...   ...  ...   
195  196   RE                        Ronald Cheng - My Only One   408   25   
196  197   RE                         YENA - SMILEY(Feat. BIBI)     4  179   
197  198  -21                         Charmaine Fong - 你是你本身的傳奇  1892   10   
198  199   RE                                    Justin Lo - 命硬  2290   30   
199  200    =  MC 張天賦 - Can’t Stop Myself Tonight - Studio Live    34  111   

      (x?) Streams Streams+     7Day    7Day+      Total  
0   

In [32]:
#import package for audtomatically changing the file names

import datetime 
from datetime import timedelta
yyyymmdd = datetime.datetime.today()
yyyymmdd = yyyymmdd-timedelta(days=2)
yyyymmdd = yyyymmdd.strftime('%Y%m%d')

In [33]:
#save files in csv. format
mydata.to_csv(f"spotify_data_{yyyymmdd}.csv",index=False)